# Heat Exchange Model

## B & V, Section 7.5

In [77]:
import numpy as np

# list of possible experiments

x_values = [-200, 0, 200]
y_values = [-300, 0, +00]
z_values = [-1, 1]

# generate all possible combinations


V = np.array([[x, y, z] for x in x_values for y in y_values for z in z_values])
p = len(V)
c = np.array([1, 1, 1])

In [78]:
import cvxpy as cvx

In [79]:
!pip install cvxpy

In [81]:
import pyomo.environ as pyo

m = pyo.ConcreteModel()

n = len(c)
p = len(V)
m.MEAS = pyo.RangeSet(n)
m.EXPTS = pyo.RangeSet(p)

m.t = pyo.Var()
m.e = pyo.Var(m.EXPTS, bounds=(-1, 1))
m.w = pyo.Var(m.EXPTS, bounds=(0, 1))

@m.Objective(sense=pyo.maximize)
def fit(m):
    return m.t

@m.Constraint(m.MEAS)
def matc(m, i):
    return m.t * c[i-1] == sum(m.e[j]*V[j-1, i-1] for j in m.EXPTS)

@m.Constraint()
def plane(m):
    return sum(m.w[i] for i in m.EXPTS) == 1

@m.Block(m.EXPTS)
def bounds(b, i):
    m = b.model()
    b.pos = pyo.Constraint(expr = m.e[i] <= m.w[i])
    b.neg = pyo.Constraint(expr = m.e[i] >= - m.w[i])
    
pyo.SolverFactory('cbc').solve(m)

for i in m.EXPTS:
    if m.w[i]() > 0.00001:
        print(f"{i:3d}, {m.e[i]():9.4f} , {m.w[i]():9.4f}", V[i-1])
    
print(m.t())

  1,   -0.0033 ,    0.0033 [-200 -300   -1]
  3,   -0.4992 ,    0.4992 [-200    0   -1]
  6,    0.4975 ,    0.4975 [-200    0    1]
1.0
